# EM Implementation

In [24]:
import numpy as np

# Set seed for reproducibility
np.random.seed(123)

# Define mean and covariance matrix for a multivariate Gaussian distribution
mean = [1, -3, 2]
cov = [[1, 0.5, 0.2], [0.5, 1, 0.3], [0.2, 0.3, 1]]

# Generate the data
data = np.random.multivariate_normal(mean, cov, size=1000)
print(data)

[[ 2.3983968  -2.06094046  1.89586593]
 [ 2.69711312 -2.68829015  3.56506145]
 [ 3.30992619 -1.69128094  3.94958089]
 ...
 [ 1.87525438 -2.20035152  2.28319017]
 [ 1.35402411 -1.65843653  2.56047618]
 [-0.92871722 -3.69263405 -0.22865864]]


In [2]:
# Introduce missing data (NaN)
missing_rate = 0.2  # 20% of the data is missing
mask = np.random.rand(*data.shape) < missing_rate
data[mask] = np.nan
print("Generated Data with Missing Values:\n", data)

Generated Data with Missing Values:
 [[ 2.3983968          nan  1.89586593]
 [ 2.69711312 -2.68829015  3.56506145]
 [ 3.30992619 -1.69128094  3.94958089]
 ...
 [        nan -2.81750546  0.88582774]
 [ 0.72765928 -3.33800423         nan]
 [ 2.459435   -2.97771047         nan]]


In [3]:
def nan_covariance(data): #not sure how to initialise without a function like this, np.cov() does not work well as there is alot of NaN in the matrix
    n, m = data.shape # this function basicaly computes covariance ignoring the mssiing data
    mean_cols = np.nanmean(data, axis=0)
    cov_matrix = np.empty((m, m))

    for i in range(m):
        for j in range(m):
            valid_data = ~np.isnan(data[:, i]) & ~np.isnan(data[:, j])
            centered_i = data[valid_data, i] - mean_cols[i]
            centered_j = data[valid_data, j] - mean_cols[j]
            cov_matrix[i, j] = np.mean(centered_i * centered_j)

    return cov_matrix

In [4]:
#Nans in data are replaced by column mean
imputed_data = np.where(np.isnan(data), np.nanmean(data, axis=0), data)
print(imputed_data)

[[ 2.3983968  -3.00348868  1.89586593]
 [ 2.69711312 -2.68829015  3.56506145]
 [ 3.30992619 -1.69128094  3.94958089]
 ...
 [ 0.98703278 -2.81750546  0.88582774]
 [ 0.72765928 -3.33800423  1.98224057]
 [ 2.459435   -2.97771047  1.98224057]]


In [5]:
def log_likelihood1(X, mu, Sigma):
    N=X.shape[0]
    p=X.shape[1]
    result=-N/2*np.log(np.linalg.det(Sigma))
    X_muT=X-mu.T
    for n in range(N):
        result=result-0.5*(X[n,:]-mu.T).dot(np.linalg.solve(Sigma, X[n,:].T-mu)) #is repetitively summing here bad?
    return result

In [6]:
def calculate_xhat(xn,mu,Sigma,missing_indices):
    #m=missing_indices.shape[0]
    p=Sigma.shape[0]
    
    xhatn=xn.copy()
    observed_indices=np.arange(p)[~np.isin(np.arange(p),missing_indices)]
    #appended_indices=np.append(observed_indices, missing_indices)
    
    Sigma22=Sigma[missing_indices,:][:,missing_indices]
    Sigma11=Sigma[observed_indices,:][:,observed_indices]
    Sigma21=Sigma[missing_indices,:][:,observed_indices]
    
    mu1=mu[observed_indices]
    mu2=mu[missing_indices]
    xhatn_1=xhatn[observed_indices]
    mu2_conditional=mu2+Sigma21.dot(np.linalg.solve(Sigma11,xhatn_1-mu1))
    #print(xhatn)
    #print(mu2_conditional,xhatn[missing_indices,],mu2_conditional.shape,xhatn[missing_indices,].shape)
    xhatn[missing_indices,] = mu2_conditional
    #this doesnt seem to work 
    #reason: Array is passed as a view, potential danger!
    #hmm still doesnt work wtf
    #Its because a numpy array of ints rounds floats that are added to it wtfffff, fixed now
    
    #print(xhatn.base)
    #print(mu2_conditional,xhatn[missing_indices])
    
    return xhatn

In [7]:
def calculate_C(Sigma,missing_indices,verbose=False):
    p=Sigma.shape[0]
    m=missing_indices.shape[0]
    if verbose:
        print("Original unconditional covariance : \n",Sigma)
    observed_indices=np.arange(p)[~np.isin(np.arange(p),missing_indices)]
    appended_indices=np.append(observed_indices, missing_indices)
    Sigma=(Sigma[appended_indices,:])[:,appended_indices]
    
    if verbose:
        print("Unconditional covariance rearranged so that missing indices are at the bottom : \n",Sigma)
    Sigma11=Sigma[0:(p-m),0:(p-m)]
    Sigma21=Sigma[(p-m):p,0:(p-m)]
    Sigma22=Sigma[(p-m):p,(p-m):p]
    Sigma22_conditional=Sigma22-Sigma21.dot(np.linalg.solve(Sigma11,Sigma21.T))
    
    reverse_permutation=np.argsort(appended_indices)
    result=np.full((p,p),0)
    result[(p-m):p,(p-m):p]=Sigma22_conditional
    if verbose:
        print("The conditional covariance when ordered is : \n",result)
    result=(result[reverse_permutation,:])[:,reverse_permutation]
    if verbose:
        print("After permuting back we get : \n", result)
    return(result)

In [20]:
def em_algorithm1(data, max_iter=100, tol=1e-6):
    n, p = data.shape

    # Initialize mean and covariance estimates
    means = np.nanmean(data, axis=0)
    covariance = nan_covariance(data)

    # Create an array to hold imputed data
    imputed_data = np.where(np.isnan(data), np.nanmean(data, axis=0), data)
    #imputed_data = np.where(np.isnan(data), None, data)
    
    old_log_likelihood = log_likelihood1(imputed_data, means, covariance)
    #print(old_log_likelihood)

    for iteration in range(max_iter):
        # E-step: Estimate missing values
        for i in range(n):
            missing = np.where(np.isnan(data[i]))[0]
            if np.isnan(data[i]).any():
                imputed_data[i,] = calculate_xhat(imputed_data[i,],means,covariance,missing) #this function works with Nans in the missing entries too, why do we impute?

        # M-step: Update mean and covariance estimates
        means = np.mean(imputed_data, axis=0)
        new_covariance = np.full((p,p),0) 

        # Add the conditional covariance for missing data
        for i in range(n):
            missing = np.where(np.isnan(data[i]))[0]
            #if missing.any(): #with this condition, are you not skipping xhat-mu *(xhat-mu).T when there is no missing data? I think so...
            new_covariance = new_covariance + calculate_C(covariance,missing,verbose=False) + (imputed_data[i,]-means).reshape(-1, 1) @ np.transpose((imputed_data[i,]-means).reshape(-1, 1))
            
        covariance = new_covariance/n 
        #print(covariance)
        # Convergence test based on log likelihood
        new_log_likelihood = log_likelihood1(imputed_data, means, covariance)
        #print(new_log_likelihood)
        difference=new_log_likelihood - old_log_likelihood
        print("The new log likelihood is :", new_log_likelihood, "  Difference of : ",difference)
        
        if np.abs(difference) < tol: #absolute value not necessary and potentially undesirable, as in theory should always be positive
            print("Convergence achieved! \n")
            break
        old_log_likelihood = new_log_likelihood

    return imputed_data,means,covariance

In [9]:
imputed_data1,mu,Sigma = em_algorithm1(data) #I don't like the name imputed_data1 for the EM-algo predictions of missing data
print("Imputed Data:\n", imputed_data1, "\n\nDifference with Old Data with mean imputation \n",imputed_data1-imputed_data)


The new log likelihood is : -9559.935349729634   Difference of :  1041.9468425925152
The new log likelihood is : -9543.07634579237   Difference of :  16.859003937264788
The new log likelihood is : -9541.805904219213   Difference of :  1.2704415731568588
The new log likelihood is : -9541.67855121083   Difference of :  0.12735300838176045
The new log likelihood is : -9541.659193707232   Difference of :  0.019357503599167103
The new log likelihood is : -9541.65551960928   Difference of :  0.003674097952170996
The new log likelihood is : -9541.654769638795   Difference of :  0.0007499704843212385
The new log likelihood is : -9541.654613505236   Difference of :  0.00015613355935784057
The new log likelihood is : -9541.654580833263   Difference of :  3.267197280365508e-05
The new log likelihood is : -9541.654573987114   Difference of :  6.8461486080195755e-06
The new log likelihood is : -9541.65457255199   Difference of :  1.4351244317367673e-06
The new log likelihood is : -9541.65457225118 

In [10]:
difference=imputed_data1-imputed_data #should not be incredibly large

print(sum(np.isnan(difference))) # No nans, good
print(sum(difference>1))
print(sum(difference>2)) #plausible enough
print(mu) #estimate is very accurate
print(Sigma)#estimate is ok but not superrr precise, especially the ones on the diagonal for example are only about 0.8
print(cov) #true covariance

[0 0 0]
[59 83  4]
[0 1 0]
[ 0.99312482 -3.00423652  1.98251282]
[[0.83906014 0.47711016 0.20636147]
 [0.47711016 0.82627886 0.31812695]
 [0.20636147 0.31812695 0.84420618]]
[[1, 0.5, 0.2], [0.5, 1, 0.3], [0.2, 0.3, 1]]


# Trying out different patterns of randomness


In [11]:
import torch

In [12]:
from produce_NA import *

C:\Users\Emil\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [25]:
X = data

# Generating missing data with diffent mechanisms
p_miss = 0.2 # missing rate
print(type(X))

#MCAR
X_mcar = produce_NA(X, p_miss=p_miss, mecha='MCAR')['X_incomp'] #So apparently this takes a numpy array but outputs sth else that must be converted
print(type(X_mcar))
#convert to np.ndarray
X_mcar = X_mcar.numpy()

#MAR, opt = "logistic", "quantile", "selfmasked"
p_obs = 0.5 # proportion of observed data 
X_mar = produce_NA(X, p_miss=p_miss, mecha='MAR', p_obs=p_obs, opt = "quantile")['X_incomp']
X_mar = X_mar.numpy()

#MNAR, opt = "logistic", "quantile", "selfmasked"
q = 0.3 # quantile at which cuts should be made
X_mnar = produce_NA(X, p_miss=p_miss, mecha='MNAR',p_obs=p_obs, q=q, opt = "logistic")['X_incomp']
#convert to np.ndarray
X_mnar = X_mnar.numpy()


<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [26]:
imputed_X_mcar,mu_mcar,Sigma_mcar=em_algorithm1(X_mcar)

imputed_X_mar,mu_mar,Sigma_mar=em_algorithm1(X_mar)

imputed_X_mnar,mu_mnar,Sigma_mnar=em_algorithm1(X_mnar)

The new log likelihood is : -906.4438067267813   Difference of :  101.90603451669654
The new log likelihood is : -900.7397138729945   Difference of :  5.704092853786847
The new log likelihood is : -899.6979859768948   Difference of :  1.041727896099701
The new log likelihood is : -899.4782444518106   Difference of :  0.2197415250841459
The new log likelihood is : -899.4263951808666   Difference of :  0.05184927094398972
The new log likelihood is : -899.4135689788073   Difference of :  0.012826202059272873
The new log likelihood is : -899.410342919679   Difference of :  0.0032260591283375106
The new log likelihood is : -899.4095270448003   Difference of :  0.0008158748787536751
The new log likelihood is : -899.4093203495108   Difference of :  0.00020669528942107718
The new log likelihood is : -899.4092679573357   Difference of :  5.2392175120985485e-05
The new log likelihood is : -899.4092546753071   Difference of :  1.3282028589856054e-05
The new log likelihood is : -899.409251308063  

In [32]:
print("True Values are :\n", mean, "\n\n",np.array(cov))
print("\nMCAR estimates are :\n",mu_mcar,"\n\n",Sigma_mcar)
print("\nMAR estimates are :\n",mu_mar,"\n\n",Sigma_mar)
print("\nMNAR estimates are :\n",mu_mnar,"\n\n",Sigma_mnar) #It even works for MNAR

True Values are :  [1, -3, 2] 
 [[1.  0.5 0.2]
 [0.5 1.  0.3]
 [0.2 0.3 1. ]]
MCAR estimates are :
 [ 0.96373355 -3.00798158  2.01511967] 

 [[0.80589378 0.42860799 0.13025841]
 [0.42860799 0.7556942  0.31719239]
 [0.13025841 0.31719239 0.84486499]]
MAR estimates are :
 [ 0.9877899  -3.02164446  1.99615656] 

 [[0.93824555 0.41939572 0.11437014]
 [0.41939572 0.75402204 0.25841325]
 [0.11437014 0.25841325 0.80146878]]
MNAR estimates are :
 [ 0.99961654 -3.01405159  1.97042697] 

 [[0.7930495  0.42466118 0.20684633]
 [0.42466118 0.76128471 0.33522432]
 [0.20684633 0.33522432 0.8393951 ]]
